In [31]:
import numpy as np
import re
import os, shutil

import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [32]:
path = '/content/dataset.txt'
with open(path, "r") as f:
  text = f.read().lower()

text = re.sub("\n", " EOS SOS ", text)
text = re.sub("\.", "", text)
text = "SOS "+text+" EOS"
dataset = word_tokenize(text)

In [33]:
#print('Number of samples:', total)
print('Number of words:', len(dataset))

Number of words: 10763


In [34]:
def generateTable(data):
    table = {}
    for i in range(len(data)-1):
        X = data[i]
        Y = data[i+1]

        if X not in table.keys():
            table[X] = {}
            table[X][Y] = 1
        else:
            if Y not in table[X].keys():
                table[X][Y] = 1
            else:
                table[X][Y] += 1
    
    return table

def convertFreqIntoProb(table):     
    for word in table.keys():
        s = float(sum(table[word].values()))
        for prob in table[word].keys():
            table[word][prob] = table[word][prob]/s
    return table

In [35]:
table = generateTable(dataset)
chain = convertFreqIntoProb(table)

In [36]:
def sample_next(word,chain):
    if word not in chain.keys():
        return "\n"
    following_words = list(chain[word].keys())
    probs = list(chain[word].values())

    mean = np.mean(probs)
    best_next = []
    for var in following_words:
        if chain[word][var] > mean:
            best_next.append(var)

    if len(best_next) == 0:
        best_next = following_words
    
    return np.random.choice(best_next)
 

In [39]:
def generateText(first_word, chain, maxLen=100):

    sentence = [first_word]
    target_word = first_word
    
    for i in range(maxLen):
        next_prediction = sample_next(target_word,chain)
        sentence.append(next_prediction)
        target_word = sentence[-1]
        if target_word == 'EOS':
            break
    
    sentence = sentence[1:]
    sentence[-1] = "."
    text = re.sub(r'\s+([?.,!’"])', r'\1', " ".join(sentence))
    return text.capitalize()

In [41]:
for i in range(20):
  print(f'{i+1}. {generateText("SOS",chain)}')

1. Your voice is always be without you and always.
2. It would have the only person i promise to the happiness in the rest of my life because you came into my life, but with your eyes, and all i was until i need to hold on the first day, i would be without you.
3. The darkness and encouragement that we will always be so much i feel alive and yet here i bet you and keep going.
4. And funny… this world.
5. Your hair and stars may not have a happy and every time i need to keep your hair and you for a mother.
6. My whole day, and love you are always learning new things and night.
7. The first sight until i would have given me? even when you were possible for all i will be impossible for that we became the love, the only one of life, so much i say that is because you know you in life, i just wanted to let you know how i promise to make you and always look great deal to tell you so many years.
8. You in life with your face when i would have been searching for all that is to hold on my whole 